In [11]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 4, something not right?

In [13]:
a = 0.25
num_params=4
num_sims = 100

In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.94 & 0.96 & 0.95 & 0.99   \\
Model 1 & 0.02 & 0.04 & 0.03 & 0.04 & 0.01   \\
Model 2 & 0.03 & 0.02 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.02, 0.03]), array([0.94, 0.04, 0.02]), array([0.96, 0.03, 0.01]), array([0.95, 0.04, 0.01]), array([0.99, 0.01, 0.  ]), 0.039604760754113164, 5.120555018067444, 5.362965951190492)


In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 0.97 & 0.97 & 0.97 & 1.00   \\
Model 1 & 0.01 & 0.03 & 0.03 & 0.03 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.99, 0.01, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([1., 0., 0.]), 1.5839740179145971, 7.002478823385885, 7.175054419109973)


# a  = .25, k= 9

In [16]:
a = 0.25
num_params=9
num_sims = 100

In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.98 & 0.99 & 0.97 & 0.99   \\
Model 1 & 0.00 & 0.01 & 0.01 & 0.02 & 0.01   \\
Model 2 & 0.04 & 0.01 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.  , 0.04]), array([0.98, 0.01, 0.01]), array([0.99, 0.01, 0.  ]), array([0.97, 0.02, 0.01]), array([0.99, 0.01, 0.  ]), -3.083635459719655, 5.063129827062749, 5.756458453464616)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.95 & 0.91 & 0.95 & 0.97   \\
Model 1 & 0.00 & 0.02 & 0.07 & 0.03 & 0.01   \\
Model 2 & 0.04 & 0.03 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.96, 0.  , 0.04]), array([0.95, 0.02, 0.03]), array([0.91, 0.07, 0.02]), array([0.95, 0.03, 0.02]), array([0.97, 0.01, 0.02]), -1.3848608882983433, 7.671064580499521, 7.754190631667735)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.86 & 0.95 & 0.95 & 0.96 & 0.98   \\
Model 1 & 0.01 & 0.03 & 0.03 & 0.01 & 0.01   \\
Model 2 & 0.13 & 0.02 & 0.02 & 0.03 & 0.01   \\
\hline
\end{tabular}
(array([0.86, 0.01, 0.13]), array([0.95, 0.03, 0.02]), array([0.95, 0.03, 0.02]), array([0.96, 0.01, 0.03]), array([0.98, 0.01, 0.01]), -4.249957580720943, 4.210220013183445, 4.220143404854845)


# a  = 0, k= 9

In [20]:
a = 0.00 
num_params=9
num_sims = 100

In [21]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.98 & 0.99 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.02 & 0.01 & 0.02 & 0.01   \\
Model 2 & 0.08 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.92, 0.  , 0.08]), array([0.98, 0.02, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.99, 0.01, 0.  ]), -3.3856216736672007, 2.019296210067682, 2.7831867871234492)


In [22]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.04 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.  , 0.04]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -3.3575049506171184, 2.0608392528496307, 2.8658773669095923)


# a  = .25

In [23]:
a = 0.25
num_params=19
num_sims = 100

In [24]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.71 & 0.94 & 0.93 & 0.94 & 0.93   \\
Model 1 & 0.00 & 0.03 & 0.02 & 0.03 & 0.03   \\
Model 2 & 0.29 & 0.03 & 0.05 & 0.03 & 0.04   \\
\hline
\end{tabular}
(array([0.71, 0.  , 0.29]), array([0.94, 0.03, 0.03]), array([0.93, 0.02, 0.05]), array([0.94, 0.03, 0.03]), array([0.93, 0.03, 0.04]), -8.783913706183807, 6.238771068428947, 6.519872162216687)


In [25]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.93 & 0.94 & 0.94 & 0.95   \\
Model 1 & 0.00 & 0.02 & 0.00 & 0.01 & 0.01   \\
Model 2 & 0.18 & 0.05 & 0.06 & 0.05 & 0.04   \\
\hline
\end{tabular}
(array([0.82, 0.  , 0.18]), array([0.93, 0.02, 0.05]), array([0.94, 0.  , 0.06]), array([0.94, 0.01, 0.05]), array([0.95, 0.01, 0.04]), -8.60489325837709, 9.014762050250035, 8.479455342227682)


In [26]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.96 & 0.96 & 0.94 & 0.96   \\
Model 1 & 0.00 & 0.02 & 0.03 & 0.04 & 0.03   \\
Model 2 & 0.11 & 0.02 & 0.01 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.89, 0.  , 0.11]), array([0.96, 0.02, 0.02]), array([0.96, 0.03, 0.01]), array([0.94, 0.04, 0.02]), array([0.96, 0.03, 0.01]), -9.046642457956013, 11.27874762692258, 11.309488696057342)


# a = 0

In [27]:
a = 0.0
num_params=19
num_sims = 100

In [28]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.41 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.59 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.41, 0.  , 0.59]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -8.841630484533678, 2.7853378694037016, 4.2356628798257985)


In [29]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.44 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.56 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.44, 0.  , 0.56]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -8.958762539909621, 3.3607774496010263, 4.28079554508181)


# evidence of power

In [30]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [31]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.78 & 0.48 & 0.49 & 0.52 & 0.59   \\
Model 1 & 0.22 & 0.52 & 0.51 & 0.48 & 0.41   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.78, 0.22, 0.  ]), array([0.48, 0.52, 0.  ]), array([0.49, 0.51, 0.  ]), array([0.52, 0.48, 0.  ]), array([0.59, 0.41, 0.  ]), 7.028849524318568, 4.72343390055436, 5.21981829708211)


In [32]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.76 & 0.46 & 0.44 & 0.46 & 0.48   \\
Model 1 & 0.24 & 0.54 & 0.56 & 0.54 & 0.52   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.76, 0.24, 0.  ]), array([0.46, 0.54, 0.  ]), array([0.44, 0.56, 0.  ]), array([0.46, 0.54, 0.  ]), array([0.48, 0.52, 0.  ]), 8.060193788702453, 5.160783074967956, 5.536732960567839)


In [33]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.17 & 0.60 & 0.61 & 0.59 & 0.68   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.83 & 0.40 & 0.39 & 0.41 & 0.32   \\
\hline
\end{tabular}
(array([0.17, 0.  , 0.83]), array([0.6, 0. , 0.4]), array([0.61, 0.  , 0.39]), array([0.59, 0.  , 0.41]), array([0.68, 0.  , 0.32]), -12.630218557351073, 4.488575688123772, 4.970380471825255)


In [34]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.20 & 0.67 & 0.68 & 0.61 & 0.72   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.80 & 0.33 & 0.32 & 0.39 & 0.28   \\
\hline
\end{tabular}
(array([0.2, 0. , 0.8]), array([0.67, 0.  , 0.33]), array([0.68, 0.  , 0.32]), array([0.61, 0.  , 0.39]), array([0.72, 0.  , 0.28]), -12.995620413568062, 4.783779250072384, 5.188220513298998)
